In [ ]:
//@version=5
indicator("4CCP + 6CBP Pattern Detector (Asia/Manila)", overlay=true, max_boxes_count=500, max_lines_count=500, max_labels_count=500)

// ────────────── Restrict to 1-Minute Timeframe
is_1m = timeframe.period == "1"

// ────────────── Candle Data
o = open
h = high
l = low
c = close

// ────────────── Helpers for Manila time formatting
f_two(n) => n < 10 ? "0" + str.tostring(n) : str.tostring(n)

// return "HH:MM" for a given bar offset (use offset = 0 for current bar, 2 for bar_index[2], etc.)
manilaTimeStr(offset) =>
    // use built-in time[] timestamp and extract hour/minute in specified timezone
    hh = hour(time[offset], "Asia/Manila")
    mm = minute(time[offset], "Asia/Manila")
    f_two(hh) + ":" + f_two(mm)

// If you prefer HH:MM:SS, replace the return with:
// ss = second(time[offset], "Asia/Manila")
// f_two(hh) + ":" + f_two(mm) + ":" + f_two(ss)

// ────────────── Utility
isBull(c_, o_) => c_ > o_
isBear(c_, o_) => c_ < o_

// ────────────── Body coords (top, bottom, mid)
getBodyCoords(_o, _c) =>
    _top = math.max(_o, _c)
    _bot = math.min(_o, _c)
    _mid = (_top + _bot) / 2
    [_top, _bot, _mid]

// ────────────── 4CCP Patterns

// 4CCP-T1.1 Bullish
t1_1 = isBull(c[3], o[3]) and
       isBear(c[2], o[2]) and c[2] < o[3] and
       isBull(c[1], o[1]) and c[1] > o[2] and c[1] > c[3] and
       isBull(c, o) and c > c[1] and c > o[2] and c > c[3]

// 4CCP-T1.2 Bullish Type 2
t1_2 = isBull(c[3], o[3]) and
       isBear(c[2], o[2]) and c[2] < o[3] and
       isBull(c[1], o[1]) and c[1] > c[2] and c[1] < o[2] and
       isBull(c, o) and c > c[1] and c > o[2] and c > c[3]

// 4CCP-T2.1 Bearish
t2_1 = isBear(c[3], o[3]) and
       isBull(c[2], o[2]) and c[2] > o[3] and
       isBear(c[1], o[1]) and c[1] < o[2] and c[1] < c[3] and
       isBear(c, o) and c < c[1] and c < o[2] and c < c[3]

// 4CCP-T2.2 Bearish Type 2
t2_2 = isBear(c[3], o[3]) and
       isBull(c[2], o[2]) and c[2] > o[3] and
       isBear(c[1], o[1]) and c[1] < c[2] and c[1] > o[2] and
       isBear(c, o) and c < c[1] and c < o[2] and c < c[3]

// ────────────── Persistent arrays
var box[] boxes = array.new_box()
var line[] lines = array.new_line()
var label[] labs = array.new_label()

// ────────────── Draw helper (WITH Manila time stamp)
drawBoxAndLine(cond, baseBarOffset, extendBars, colorFill) =>
    if is_1m and cond
        [top, bot, mid] = getBodyCoords(open[baseBarOffset], close[baseBarOffset])
        x1 = bar_index - baseBarOffset
        x2 = x1 + extendBars

        bx = box.new(left = x1, right = x2, top = top, bottom = bot,
                     bgcolor = color.new(colorFill, 70), border_color = colorFill)
        ln = line.new(x1 = x1, y1 = mid, x2 = x2, y2 = mid,
                      color = colorFill, width = 1, style = line.style_dotted)

        array.push(boxes, bx)
        array.push(lines, ln)  

        // Manila timestamp for the pattern bar
        ts = manilaTimeStr(baseBarOffset)
        tLabel = label.new( x1, top,ts, style = label.style_label_down,textcolor = color.rgb(0, 0, 0),  color = color.new(colorFill, 100),size = size.small )
        array.push(labs, tLabel)

// ────────────── Draw all patterns
drawBoxAndLine(t1_1, 2, 5, color.rgb(0, 192, 255))
drawBoxAndLine(t1_2, 2, 5, color.rgb(0, 38, 255))
drawBoxAndLine(t2_1, 2, 5, color.rgb(255, 64, 64))
drawBoxAndLine(t2_2, 2, 5, color.rgb(255, 128, 0))

// ────────────────────────────
// Helper Function: Midpoint
get_mid(_o, _c) => (_o + _c) / 2

// ────────────────────────────
// 6CBP Logic
bear_bull1 = c[5] > o[5]
bear_bull2 = c[4] > o[4] and c[4] > c[5]
bear_bear3 = c[3] < o[3] and c[3] < o[4]
bear_bull4 = c[2] > c[3] and c[2] < o[3]
bear_bear5 = c[1] < o[2] and c[1] < c[3] and c[1] < o[4]
bear_bear6 = c < o and c < o[5] and c < c[1]

bearish_6CBP = bear_bull1 and bear_bull2 and bear_bear3 and bear_bull4 and bear_bear5 and bear_bear6

o4_bear = o[2]
c4_bear = c[2]
h4_bear = h[2]
l4_bear = l[2]
mid4_bear = get_mid(o4_bear, c4_bear)

// Bullish inverse
bull_bear1 = c[5] < o[5]
bull_bear2 = c[4] < o[4] and c[4] < c[5]
bull_bull3 = c[3] > o[3] and c[3] > o[4]
bull_bear4 = c[2] < c[3] and c[2] > o[3]
bull_bull5 = c[1] > o[2] and c[1] > c[3] and c[1] > o[4]
bull_bull6 = c > o and c > o[5] and c > c[1]

bullish_6CBP = bull_bear1 and bull_bear2 and bull_bull3 and bull_bear4 and bull_bull5 and bull_bull6

o4_bull = o[2]
c4_bull = c[2]
h4_bull = h[2]
l4_bull = l[2]
mid4_bull = get_mid(o4_bull, c4_bull)

// ────────────────────────────
// 🟥 Bearish 6CBP
if bearish_6CBP
    // Tiny diamond on top wick of 4th candle
    label.new(bar_index[2], h4_bear, "6CBP↓",  style=label.style_diamond, color=color.new(color.red, 0), textcolor=color.new(#000000, 0), size=size.tiny)

    // Rectangle for 4th candle’s body, extended 10 bars right
    box.new(left=bar_index[2], right=bar_index[2] + 10, top=math.max(o4_bear, c4_bear), bottom=math.min(o4_bear, c4_bear),bgcolor=color.new(color.red, 85),border_color=color.new(color.red, 0))

    // Midline for reference
    line.new(bar_index[2], mid4_bear, bar_index[2] + 10, mid4_bear,  color=color.new(color.red, 0), width=1 )

    // ✅ Manila timestamp for this 4th-bar (offset = 2)
    label.new(bar_index[2], h4_bear, manilaTimeStr(2), style = label.style_label_down,  textcolor = color.rgb(0, 0, 0), color = color.new(color.red, 100),  size = size.small)

// ────────────────────────────
// 🟩 Bullish 6CBP
if bullish_6CBP
    // Tiny diamond on bottom wick of 4th candle
    label.new(bar_index[2], l4_bull, text="6CBP↑",style=label.style_diamond,color=color.new(color.lime, 0),textcolor=color.new(#000000, 0), size=size.tiny )

    // Rectangle for 4th candle’s body, extended 10 bars right
    box.new(left=bar_index[2], right=bar_index[2] + 10, top=math.max(o4_bull, c4_bull),bottom=math.min(o4_bull, c4_bull), bgcolor=color.new(color.lime, 85),    border_color=color.new(color.lime, 0)  )

    // Midline for reference
    line.new(  bar_index[2],  mid4_bull,  bar_index[2] + 10,  mid4_bull, color=color.new(color.lime, 0), width=1  )

    // ✅ Manila timestamp for this 4th-bar (offset = 2)
    label.new(  bar_index[2], l4_bull, manilaTimeStr(2),style = label.style_label_up, textcolor = color.rgb(0, 0, 0),color = color.new(color.lime, 100),  size = size.small)
